In [26]:
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

import pandas as pd

from scipy import optimize
from scipy import integrate

In [2]:
# url from the Covid-19 data is extracted
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"

In [3]:
def get_data(url):
    
    """
    Reads CSV data COVID-19 data of different counties from the url  
    
    INPUT: URL of the CSV
    OUTPUT: Pandas dataframe

    """
    
    url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
    covid_data = pd.read_csv(url,sep=",")
    print(f"The number of unique countries in the data are {len(covid_data['location'].unique())}")
    
    return covid_data
    

In [12]:
def clean_data(df):
    """
    Takes in the dataframe and
    - removes unnecessary columns
    - fills the NaN values with zeroes [Assumption: NaN means there were no cases that day instead of filling it with mean values] 
    - correcting the datetime format
    
    INPUT: Pandas dataframe
    OUTPUT: Cleaned dataframe

    """
    cases = df[['date','location','new_cases_smoothed','total_cases', 'total_deaths', 'new_deaths_smoothed']]
    cases = cases.fillna(0)
    
    # converting date object into datetime
    cases["date"]= pd.to_datetime(cases["date"])
    
    # reseting the index
    cases = cases.reset_index(drop = True)
    cases['new_cases_smoothed'] = cases['new_cases_smoothed'].apply(np.ceil)
    
    loc = "E:/uni/Data_Science_E/Data_Science_Enterprise/data/final_cases.csv"
    # saving the dataset
    cases.to_csv(loc,index=False)
    
    return cases
    

In [13]:
def prep_fitting_data(df,countries):
    new = df[df.location.isin(countries)]
    table = pd.pivot_table(new, values='new_cases_smoothed', index=['date'],
                        columns=['location'], fill_value=0)
    loc = "E:/uni/Data_Science_E/Data_Science_Enterprise/data/table.csv"
    # saving the dataset
    table.to_csv(loc,index=False)
    
    return table

In [29]:
covid_data = get_data(url)
cases = clean_data(covid_data)

The number of unique countries in the data are 244


In [56]:
N0=1000000 #max susceptible population
beta=0.4   # infection spread dynamics/contact rate
gamma=0.1  # recovery rate
t=np.arange(cases.Germany[220:320])
I0=25
S0=N0-I0
R0=0

AttributeError: 'DataFrame' object has no attribute 'Germany'

In [46]:
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [45]:
def fit_odeint( x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

In [30]:
countries = ['Germany','Italy','France']

In [49]:
app = dash.Dash()

app.layout = html.Div([
    html.H4("Fitting SIR model"),
    html.P("Select Country:"),
    dcc.Dropdown(
        id='dropdown',
        options=["Germany", "Italy", "France"],
        value='Germany',
        multi=False
    ),
    dcc.Graph(id="graph"),
])


@app.callback(
    Output("graph", "figure"), 
    Input('dropdown', "value"))

def fit_and_display(name): 
    
    df = prep_fitting_data(cases,countries)
    
    # set some basic parameters
    # beta/gamma is denoted as  'basic reproduction number'

        
    if name == "Germany":
        ydata = df.Germany[220:320]
    elif name == "Italy":
        ydata = df.Italy[220:320]
    else:
        ydata = df.France[220:320]
    
    t=np.arange(len(ydata))
    
    # ensure re-initialization 
    I0=ydata[0]
    S0=N0-I0
    R0=0
    
    # example curve of our differential equationa
    popt=[0.4,0.1]
    fit_odeint(t, *popt)
    
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
    perr = np.sqrt(np.diag(pcov))

    print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
    print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
    
     # get the final fitted curve
    fitted=fit_odeint(S0,I0,R0,t, *popt)
        
    
    fig = go.Figure([
        go.Scatter(x=t, y=ydata, 
                   name='true cases', mode='markers'),
        go.Scatter(x=t, y=fitted, 
                   name='prediction')
    ])
    return fig


In [57]:
if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
